This process creates a unique file from the Adtech Dashboard Weekly Exports pulled from the <a ref='https://app.powerbi.com/groups/me/apps/638779b4-2f48-4b27-b5e4-cf8f63f8eff7/reports/f3ebd76a-8e1b-4791-9311-9f8ed418b04d/ReportSection53f989641da9e91409d0?ctid=945c199a-83a2-4e80-9f8c-5a91be5752dd&experience=power-bi'>Adtech dashboard</a>.

<h2>Library</h2>

In [14]:
import warnings
import os
import re
import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

In [15]:
def remove_typos_from_dataframe(df, column_name):
    # Define a list of common typos and their corrections
    typos = {
        '3PD AUDIENCE' : '3PD AUDIENCE',
        'ABM-1PD AUDIENCE' : 'ABM-1PD AUDIENCE',
        'ABM-3PD AUDIENCE' : 'ABM-3PD AUDIENCE',
        'ABM-3PD AUDIENMB' : 'ABM-3PD AUDIENCE',
        'ABM-3PD INDIENCE' : 'ABM-3PD AUDIENCE',
        'ABM-3PD JPDIENCE' : 'ABM-3PD AUDIENCE',
        'AFFINITY -1PD PIXEL' : 'AFFINITY-1PD PIXEL',
        'AFFINITY-1PD PIXEL' : 'AFFINITY-1PD PIXEL',
        'AFFINITY-3PD   BEHAVIOR' : 'AFFINITY-3PD BEHAVIOR',
        'AFFINITY-3PD BEHAVIOR' : 'AFFINITY-3PD BEHAVIOR',
        'AFFINITY-3PDÂ BEHAVIOR' : 'AFFINITY-3PD BEHAVIOR',
        'AFFINITY-3PD-BEHAVIOR' : 'AFFINITY-3PD BEHAVIOR',
        'AUDIO-3PD RETARGET EXPOSED' : 'AUDIO-3PD RETARGET EXPOSED',
        'BEHAVIOR-3PD' : 'BEHAVIORAL-3PD',
        'BEHAVIORAL-3PD' : 'BEHAVIORAL-3PD',
        'BEHAVIORAL-3PD RETARGET' : 'BEHAVIORAL-3PD RETARGET',
        'CAMPAIGN-3PD LAL' : 'CAMPAIGN-3PD LAL',
        'CAMPAIGN-3PD-LAL' : 'CAMPAIGN-3PD LAL',
        'CATEGORY-3PD CON CONTEXTUAL' : 'CATEGORY-3PD CONTEXTUAL',
        'CATEGORY-3PD CONTEXTUAL' : 'CATEGORY-3PD CONTEXTUAL',
        'CATEGORY-3PD CONTEXTUALL' : 'CATEGORY-3PD CONTEXTUAL',
        'CATEGORY-3PD CONTEXTUALT' : 'CATEGORY-3PD CONTEXTUAL',
        'CATEGORY-3PDÂ CONTEXTUAL' : 'CATEGORY-3PD CONTEXTUAL',
        'CATEGORY-3PD-CONTEXTUAL' : 'CATEGORY-3PD CONTEXTUAL',
        'CE Affinity-3PD BEHAVIOR' : 'CE AFFINITY-3PD BEHAVIOR',
        'CONTEXTUAL-3PD' : 'CONTEXTUAL-3PD',
        'CONTEXTUAL-3PD RETARGET' : 'CONTEXTUAL-3PD RETARGET',
        'CRM-1PD' : 'CRM-1PD',
        'CRM-1PD AUDIENCE' : 'CRM-1PD AUDIENCE',
        'CRM-1PD CRM' : 'CRM-1PD CRM',
        'CRM-1PDÂ CRM' : 'CRM-1PD CRM',
        'CRM-1PD-AIQ-NS' : 'CRM-1PD-AIQ-NS',
        'CRM-3PD LAL' : 'CRM-3PD LAL',
        'CTV-3PD RETARGET EXPOSED' : 'CTV-3PD RETARGET EXPOSED',
        'DIGITAL-3PD RETARGET EXPOSED' : 'DIGITAL-3PD RETARGET EXPOSED',
        'DMP-3PD  AUDIENCE' : 'DMP-3PD AUDIENCE',
        'DMP-3PD ANZDIENCE' : 'DMP-3PD AUDIENCE',
        'DMP-3PD AUDIENCE' : 'DMP-3PD AUDIENCE',
        'DMP-3PD AUDIFRCE' : 'DMP-3PD AUDIENCE',
        'DMP-3PD INDIENCE' : 'DMP-3PD AUDIENCE',
        'DMP-3PD JPDIENCE' : 'DMP-3PD AUDIENCE',
        'DMP-3PD LAL' : 'DMP-3PD LAL',
        'DMP-3PD SGDIENCE' : 'DMP-3PD AUDIENCE',
        'DMP-3PDÂ AUDIENCE' : 'DMP-3PD AUDIENCE',
        'DOMAIN-3PD CONTEXTUAL' : 'DOMAIN-3PD CONTEXTUAL',
        'DOMAIN-3PDÂ CONTEXTUAL' : 'DOMAIN-3PD CONTEXTUAL',
        'EMAIL-1PD CRM' : 'EMAIL-1PD CRM',
        'IN   MARKET-3PD BEHAVIOR' : 'IN MARKET-3PD BEHAVIOR',
        'IN  MARKET-3PD CONTEXTUAL' : 'IN MARKET-3PD CONTEXTUAL',
        'IN MARKET -1PD PIXEL' : 'IN MARKET-1PD PIXEL',
        'IN MARKET- 3PD BEHAVIOR' : 'IN MARKET-3PD BEHAVIOR',
        'IN MARKET-1PD PIXEL' : 'IN MARKET-1PD PIXEL',
        'IN MARKET-3PD  BEHAVIOR' : 'IN MARKET-3PD BEHAVIOR',
        'IN MARKET-3PD  CONTEXTUAL' : 'IN MARKET-3PD CONTEXTUAL',
        'IN MARKET-3PD BEHAVIOR' : 'IN MARKET-3PD BEHAVIOR',
        'IN MARKET-3PD BEHAVIORVID IN-STREAM' : 'IN MARKET-3PD BEHAVIOR',
        'IN MARKET-3PD BEHAVIOUR' : 'IN MARKET-3PD BEHAVIOR',
        'IN MARKET-3PD CONTEXTUAL' : 'IN MARKET-3PD CONTEXTUAL',
        'INÂ MARKET-3PDÂ BEHAVIOR' : 'IN MARKET-3PD BEHAVIOR',
        'INÂ MARKET-3PDÂ CONTEXTUAL' : 'IN MARKET-3PD CONTEXTUAL',
        'IN-MARKET-3PD BEHAVIOR' : 'IN-MARKET-3PD BEHAVIOR',
        'KCT-3PD CONTEXTUAL' : 'KCT-3PD CONTEXTUAL',
        'KSTD-3PD CONTEXTUAL' : 'KSTD-3PD CONTEXTUAL',
        'LAL-3PD BEHAVIOR' : 'LAL-3PD BEHAVIOR',
        'LAPSED - 3PD BEHAVIOR' : 'LAPSED-3PD BEHAVIOR',
        'LAPSED LAL-1PD PIXEL' : 'LAPSED LAL-1PD PIXEL',
        'Lapsed User-3PD BEHAVIOR' : 'LAPSED USER-3PD BEHAVIOR',
        'LAPSED USERS -1PD PIXEL' : 'LAPSED USERS-1PD PIXEL',
        'LAPSED USERS-1PD PIXEL' : 'LAPSED USERS-1PD PIXEL',
        'LAPSED-1PD PIXEL' : 'LAPSED-1PD PIXEL',
        'LAPSED-1PD PIXEL-1PD PIXEL' : 'LAPSED-1PD PIXEL-1PD PIXEL',
        'LAPSED-1PDÂ PIXEL' : 'LAPSED-1PD PIXEL',
        'MB Affinity-3PD BEHAVIOR' : 'MB AFFINITY-3PD BEHAVIOR',
        'MEDIA-1PD' : 'MEDIA-1PD',
        'MEDIA-1PD MEDIA' : 'MEDIA-1PD MEDIA',
        'MEDIA-1PDÂ MEDIA' : 'MEDIA-1PD MEDIA',
        'MEDIA-3PD RETARGET EXPOSED' : 'MEDIA-3PD RETARGET EXPOSED',
        'PERSONA-1PD RETARGET' : 'PERSONA-1PD RETARGET',
        'PIXEL-3PD LAL' : 'PIXEL-3PD LAL',
        'PIXEL-3PD LAL-Family' : 'PIXEL-3PD LAL-FAMILY',
        'PIXEL-3PD LAL-Product' : 'PIXEL-3PD LAL-PRODUCT',
        'PIXEL-3PDÂ LAL' : 'PIXEL-3PD LAL',
        'PROXIMITY-3PD BEHAVIOR' : 'PROXIMITY-3PD BEHAVIOR',
        'PROXIMITY-3PD CONTEXTUAL' : 'PROXIMITY-3PD CONTEXTUAL',
        'RETARGET-1PD   RETARGET' : 'RETARGET-1PD RETARGET',
        'RETARGET-1PD  RETARGET' : 'RETARGET-1PD RETARGET',
        'RETARGET-1PD RETARGET' : 'RETARGET-1PD RETARGET',
        'RETARGET-1PDÂ RETARGET' : 'RETARGET-1PD RETARGET',
        'RETARGET-3PD COM SIZE' : 'RETARGET-3PD COM SIZE',
        'Retargeting-3PD PIXEL' : 'RETARGETING-3PD PIXEL',
        'ROS-3PD PMP' : 'ROS-3PD PMP',
        'ROS-3PD ROS' : 'ROS-3PD ROS',
        'SITE IN MARKET-3PD BEHAVIOR' : 'SITE IN MARKET-3PD BEHAVIOR',
        'SITE IN MARKET-3PD CONTEXTUAL' : 'SITE IN MARKET-3PD CONTEXTUAL',
        'VIDEO-3PD RETARGET EXPOSED' : 'VIDEO-3PD RETARGET EXPOSED',
    }

    # Iterate through the DataFrame and apply typo corrections
    for index, row in df.iterrows():
        if row[column_name] in typos:
            df.at[index, column_name] = typos[row[column_name]]

    return df

<h2>Main</h2>

<h2>B2B File</h2>

In [16]:
folder_path = r'C:\Users\Rafael_Fagundes\Downloads\B2B'
all_data_b2b = []

file_list = os.listdir(folder_path)

for file_name in file_list:
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        try:
            df_b2b = pd.read_excel(file_path)
            all_data_b2b.append(df_b2b)
        except Exception as e:
            print(f"Error reading '{file_name}': {e}")

if all_data_b2b:
    df_b2b_contact = pd.concat(all_data_b2b, ignore_index=True)
else:
    print("No valid Excel files found in the specified folder.")

In [17]:
df_b2b_contact.drop(columns={'[SumVisits]','[v__Campaings]','[M3]'}, inplace=True)
df_b2b_contact.rename(columns={
    'Data[Global Audience Type (Calc)]':'Audience Type'
    ,'Data[B2B Tactic]':'Tactic'
    ,'Data[Media Buy Name]':'Media Buy Name'
    ,'Data[Campaign Name]':'Campaign Name'
    ,'Data[Fiscal Quarter]':'Fiscal Quarter'
    ,'Data[Fiscal Week]':'Fiscal Week'
    ,'[SumImpressions]':'Impressions'
    ,'[SumClicks]':'Clicks'
    ,'[SumSpend]':'Spend'
    ,'[SumNet_Rev_B2B]':'Net Rev'
    ,'[SumHVE_B2B]':'HVE'
    ,'Data[Country]':'Country'
    ,'Data[Display Funnel Mapped]':'Display Funnel Mapped'
    ,'Data[Display Dell Vehicle Mapped]':'Display Dell Vehicle Mapped'
    ,'Data[Display Business Unit Mapped]':'Display Business Unit Mapped'
    ,'Data[Display Site Name Mapped]':'Display Site Name Mapped'
    ,'Data[Audience Type Name]':'Audience Type Name'
} , inplace=True)

df_b2b_contact['BU'] = 'B2B'

<h2>CSB & B2B Merge</h2>

In [18]:
# Merge CSB & B2B dataframes & remove duplicates
df_merged = df_b2b_contact.drop_duplicates()

# df_merged.drop_duplicates(inplace=True)

In [19]:
# Get Claravine ID: _d1n4f7g4t1i_
df_merged['Segment ID'] = df_merged['Media Buy Name'].replace('-', '_', regex=True).str.extract('(_d[a-zA-Z0-9]{10}_)')

df_merged['Segment ID'] = df_merged['Segment ID'].str.replace('_', '')

df_merged.fillna(0, inplace=True)

df_merged['Audience ID Exists'] = df_merged['Segment ID'].apply(lambda x: 'No ID' if x == 0 else 'Have ID')

<h2>Claravine Merge</h2>

In [20]:
# Read Claravine file
df_claravine = pd.read_csv('csv\Claravine.csv')

In [21]:
df_final = pd.merge(df_merged, df_claravine, on ='Segment ID', how='left')

df_final.fillna(0, inplace=True)

<h2>Compile Data Export</h2>

In [22]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904290 entries, 0 to 904289
Data columns (total 48 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   Audience Type                                904290 non-null  object 
 1   Tactic                                       904290 non-null  object 
 2   Media Buy Name                               904290 non-null  object 
 3   Campaign Name                                904290 non-null  object 
 4   Fiscal Quarter                               904290 non-null  object 
 5   Fiscal Week                                  904290 non-null  object 
 6   Country_x                                    904290 non-null  object 
 7   Display Funnel Mapped                        904290 non-null  object 
 8   Display Dell Vehicle Mapped                  904290 non-null  object 
 9   Display Business Unit Mapped                 904290 non-nul

<h2>Fix [Audience Type Nam] Typos</h2>

In [23]:
df_final = remove_typos_from_dataframe(df_final, 'Audience Type Name')

df_final['Audience Type Name'] = df_final['Audience Type Name'].str.replace('\u00A0', ' ')

In [24]:
folder_path = r'C:\Users\Rafael_Fagundes\Downloads'

df_final.to_csv(folder_path + r'\b2b_compiled_data.csv')